<a href="https://colab.research.google.com/github/mlindsey19/EvolComp-Rucksack/blob/master/Rucksack_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

float_formatter = lambda x: "%.0f" % x
float_formatter2 = lambda x:  str(x)[0] if ( 0 <= x ) else str(x)[0:2] #for veiwing aligned columns. ONLY SHOWS FIRST DIGIT
np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )

L = 20
N = 10

bits = np.random.randint(low=0,high=2,size=20)
def new_bits():
  global bits
  bits = np.random.randint(low=0,high=2,size=20)
  return bits # need this return
  
vals = np.random.normal(loc=10.0, scale=2.0, size=20)
wts = np.random.normal(loc=8.0, scale=3.0, size=20)
ratio = vals/wts
cons = np.stack( [ vals, wts, ratio ] )

avg_c = wts.sum()/2
restrict_c = wts.max() * 2

def new_constraits():
  global avg_c,restict_c, vals, wts, cons
  vals = np.random.normal(loc=10.0, scale=2.0, size=20)
  wts = np.random.normal(loc=8.0, scale=3.0, size=20)
  ratio = vals/wts
  cons = np.stack( [ vals, wts, ratio ] )
  avg_c = wts.sum()/2
  restrict_c = wts.max() * 2

pop = [ np.stack( [ new_bits(), vals, wts, ratio ]) for _ in range(N) ]
pop = np.stack(pop)

def new_pop():
  global pop
  pop = [ np.stack( [ new_bits(), vals, wts, ratio ]) for _ in range(N) ]
  pop = np.stack(pop)
  
def new_problem():
  new_constraits()
  new_pop()



In [32]:
def str_wt( x ):
  return ( x[0] * x[2] ).sum()
  
print(str_wt(pop[4]))

80.05197207451062


* **x** -> individual
* **c** -> capacity 
* **d** -> degree of penalty ( logarithmic = 0;  linear = 1; quadratic = 2 )





In [36]:
pen = ratio.max()
def set_pen():
  global pen
  pen = ratio.max()


def penalty(x, c, d ):
  global pen
  p = ( str_wt( x ) - c) * pen
  p = np.log2( 1 + p) if ( d == 0 ) else p #assumes only positive values of p
  p = p*p if (d == 2) else p
  return p

penalty(pop[3], avg_c, 1)

-16.347655987103767

#Repair


In [0]:


def repair(x, c):
  overfill = True if ( str_wt( x ) > c ) else False
  while( overfill == True):
    
    

In [14]:
  ratio


array([1.98, 1.46, 2.89, 10.82, 1.35, 2.74, 2.77, 1.10, 1.51, 1.11, 1.09,
       1.24, 1.80, 0.79, 1.36, 1.09, 0.90, 0.84, 1.59, 6.37])

In [9]:
vals

array([12.64, 9.57, 12.99, 6.98, 10.93, 11.48, 10.15, 9.26, 13.05, 9.67,
       9.33, 11.54, 6.33, 5.71, 9.80, 7.35, 12.16, 11.40, 9.48, 9.73])

In [10]:
wts

array([4.04, 4.90, 7.84, 5.48, 7.40, 7.27, 16.57, 6.54, 6.26, 5.10, 5.26,
       8.36, 10.23, 11.22, 2.21, 9.43, 6.60, 10.30, 10.48, 9.52])

In [11]:
avg_c

77.4941922052184

In [12]:
restrict_c

33.14479241907637